[실습] titanic3.csv 파일로 생존자 예측 모델 생성을 위한 분석용 데이터 셋을 작성하세요.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


#데이터 불러오기
file_path= '/content/drive/MyDrive/kdt_240424/m3_분석라이브러리/pandas/dataset/titanic3.csv' #드라이브 마운트 불러오기.
df = pd.read_csv(file_path,index_col=0)

print(df.head())

        survived                                             name     sex  \
pclass                                                                      
1              1                    Allen, Miss. Elisabeth Walton  female   
1              1                   Allison, Master. Hudson Trevor    male   
1              0                     Allison, Miss. Helen Loraine  female   
1              0             Allison, Mr. Hudson Joshua Creighton    male   
1              0  Allison, Mrs. Hudson J C (Bessie Waldo Daniels)  female   

          age  sibsp  parch  ticket      fare    cabin embarked boat   body  \
pclass                                                                        
1       29.00      0      0   24160  211.3375       B5        S    2    NaN   
1        0.92      1      2  113781  151.5500  C22 C26        S   11    NaN   
1        2.00      1      2  113781  151.5500  C22 C26        S  NaN    NaN   
1       30.00      1      2  113781  151.5500  C22 C26        S  

In [ ]:
# 결측치 확인
print(df.isnull().sum())

survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64


In [ ]:
# 필요없는 열 제거
df.drop(columns=['name', 'ticket', 'body', 'home.dest', 'boat','cabin'], inplace=True)
df.head()

,survived,sex,age,sibsp,parch,fare,embarked
pclass,,,,,,,
1,1,female,29.00,0,0,211.3375,S
1,1,male,0.92,1,2,151.5500,S
1,0,female,2.00,1,2,151.5500,S
1,0,male,30.00,1,2,151.5500,S
1,0,female,25.00,1,2,151.5500,S


In [ ]:
# 결측치 처리
df['age'].fillna(df['age'].mean(), inplace = True)
df['fare'].fillna(df['fare'].mean(), inplace = True)
df['embarked'].fillna(df['embarked'].mode()[0], inplace = True)
# df['pclass'].fillna(df['pclass].mode()[0], inplcae=True)

In [ ]:
# age_group을 6개 범주로 확대
bins = [0,5,12,18,35,60,100]
labels = ['Infant', 'Child', 'Teenager', 'Young Adult', 'Adult', 'Senior']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels)
df.drop(columns=['age'], inplace=True)

In [ ]:
# fare를 3개 범주로 변환
fare_bins = [0,30,100,600]
fare_labels = ['Low','Medium','High']
df['fare_group'] = pd.cut(df['fare'], bins=fare_bins, labels=fare_labels)
df.drop(columns = ['fare'], inplace=True)

In [ ]:
# sibsp와 parch 합쳐서 가족 규모 변수 생성
df['family_size'] = df['sibsp'] + df['parch'] + 1
df.drop(columns=['sibsp', 'parch'], inplace=True)

In [ ]:
# 범주형 변수를 더미 변수로 변환할 열 목록
categorical_columns = ['age_group', 'fare_group', 'sex', 'embarked']

# 반복문을 사용하여 범주형 변수를 더미 변수로 변환
for column in categorical_columns:
    # 첫 번째 카테고리는 drop_first=True로 제거
    df = pd.get_dummies(df, columns=[column], drop_first=True)

In [ ]:
df.head()

,survived,family_size,age_group_Child,age_group_Teenager,age_group_Young Adult,age_group_Adult,age_group_Senior,fare_group_Medium,fare_group_High,sex_male,embarked_Q,embarked_S
pclass,,,,,,,,,,,,
1,1,1,False,False,True,False,False,False,True,False,False,True
1,1,4,False,False,False,False,False,False,True,True,False,True
1,0,4,False,False,False,False,False,False,True,False,False,True
1,0,4,False,False,True,False,False,False,True,True,False,True
1,0,4,False,False,True,False,False,False,True,False,False,True


In [ ]:
# 타겟 변수와 피처 변수 설정
X = df.drop('survived', axis=1)
y = df['survived']

# 학습 데이터와 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 크기 확인
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 성능 평가
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


(1047, 11) (262, 11) (1047,) (262,)
Accuracy: 0.767175572519084
Confusion Matrix:
 [[123  21]
 [ 40  78]]
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.85      0.80       144
           1       0.79      0.66      0.72       118

    accuracy                           0.77       262
   macro avg       0.77      0.76      0.76       262
weighted avg       0.77      0.77      0.76       262

